<a href="https://colab.research.google.com/github/GitDarrwin/FYP/blob/main/model60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: connect to drive

from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [1]:
import tarfile
import zipfile

with tarfile.open('/content/drive/MyDrive/Colab Notebooks/Dataset for MALWARE/label_metadata.tar.gz', 'r:gz') as tar:
  tar.extractall('/content/extracted_files/label') # Extract to a new directory

with tarfile.open('/content/drive/MyDrive/Colab Notebooks/Dataset for MALWARE/malnet-graphs-tiny.tar.gz', 'r:gz') as tar:
  tar.extractall('/content/extracted_files/malnet') # Extract to the same directory

with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Dataset for MALWARE/split_info_tiny.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/extracted_files/splitinfo') # Extract to the same directory

In [2]:
!pip install gensim

In [3]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.7 MB/s eta 0:00:00


In [ ]:
# import os
# import random
# import numpy as np
# import torch
# import torch.nn.functional as F
# from torch.optim import AdamW
# from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
# from tqdm import tqdm
# import networkx as nx
# from torch_geometric.data import Data, DataLoader
# from torch_geometric.nn import GATConv, global_mean_pool
# from sklearn.metrics import classification_report, accuracy_score, f1_score
# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE
# import yt_dlp
# import IPython.display as ipd

# def play_youtube_audio(url):
#     ydl_opts = {
#         'format': 'bestaudio/best',
#         'postprocessors': [{
#             'key': 'FFmpegExtractAudio',
#             'preferredcodec': 'mp3',
#             'preferredquality': '192',
#         }],
#         'outtmpl': 'audio.%(ext)s'
#     }
#     with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#         ydl.download([url])
#     return ipd.Audio("audio.mp3", autoplay=True)

# # --- Set seeds and device ---
# SEED = 75
# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(SEED)
#     torch.backends.cudnn.deterministic = True
#     # Enable benchmark for faster training when possible
#     torch.backends.cudnn.benchmark = True

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Using device: {device}")

# # --- Label and path definitions ---
# LABEL_MAPPING = {
#     'adware': 0,
#     'benign': 1,
#     'downloader': 2,
#     'trojan': 3,
#     'addisplay': 4  # This class will be treated as unseen during training
# }

# # Set file paths (modify if needed)
# graph_data_path = '/content/extracted_files/malnet/malnet-graphs-tiny'
# split_info_path = '/content/extracted_files/splitinfo/split_info_tiny/type'

# # --- Split load helper ---
# def load_split(file_name):
#     split_file = os.path.join(split_info_path, file_name)
#     with open(split_file, 'r') as f:
#         return [line.strip() for line in f]

# train_graphs = load_split('train.txt')
# val_graphs = load_split('val.txt')
# test_graphs = load_split('test.txt')

# # --- Node feature calculation ---
# @torch.no_grad()
# def calculate_node_features(G_nx, num_nodes):
#     # Existing features
#     degrees = torch.tensor([d for _, d in G_nx.degree()], dtype=torch.float32)
#     G_undirected = G_nx.to_undirected()
#     clustering_dict = nx.clustering(G_undirected)
#     clustering_values = torch.tensor([clustering_dict[i] for i in range(num_nodes)], dtype=torch.float32)

#     # Calculate additional centrality metrics
#     # For larger graphs, use approximation to improve speed
#     if num_nodes > 500:
#         betweenness_dict = nx.betweenness_centrality(G_nx, k=min(50, num_nodes))
#         closeness_dict = {i: 0.5 for i in range(num_nodes)}  # Default value for large graphs
#     else:
#         betweenness_dict = nx.betweenness_centrality(G_nx)
#         try:
#             closeness_dict = nx.closeness_centrality(G_nx)
#         except:
#             closeness_dict = {i: 0.5 for i in range(num_nodes)}

#     betweenness_values = torch.tensor([betweenness_dict[i] for i in range(num_nodes)], dtype=torch.float32)
#     closeness_values = torch.tensor([closeness_dict[i] for i in range(num_nodes)], dtype=torch.float32)

#     # Calculate eigenvector centrality (important for capturing influence patterns)
#     try:
#         eigenvector_dict = nx.eigenvector_centrality(G_nx, max_iter=100)
#     except Exception as e:
#         eigenvector_dict = {i: 1.0 / num_nodes for i in range(num_nodes)}
#     eigenvector_values = torch.tensor([eigenvector_dict[i] for i in range(num_nodes)], dtype=torch.float32)

#     # Calculate pagerank (useful for identifying important nodes)
#     try:
#         pagerank_dict = nx.pagerank(G_nx, max_iter=100)
#     except:
#         pagerank_dict = {i: 1.0 / num_nodes for i in range(num_nodes)}
#     pagerank_values = torch.tensor([pagerank_dict[i] for i in range(num_nodes)], dtype=torch.float32)

#     # Include graph density feature (trojans often have distinct density patterns)
#     graph_density = nx.density(G_nx)
#     density_feature = torch.full((num_nodes, 1), graph_density, dtype=torch.float32)

#     return torch.cat([
#         degrees.unsqueeze(1),
#         clustering_values.unsqueeze(1),
#         betweenness_values.unsqueeze(1),
#         eigenvector_values.unsqueeze(1),
#         closeness_values.unsqueeze(1),
#         pagerank_values.unsqueeze(1),
#         density_feature
#     ], dim=1)

# # --- Graph loading function with improved error handling ---
# def load_graphs(graph_list, desc="Loading graphs"):
#     data_list = []
#     with tqdm(total=len(graph_list), desc=desc, position=0, leave=True) as pbar:
#         for graph_entry in graph_list:
#             # Expecting "class/filename" structure
#             category_folder, filename = os.path.split(graph_entry)
#             class_name = category_folder.split('/')[0]
#             if class_name not in LABEL_MAPPING:
#                 pbar.update(1)
#                 continue

#             graph_path = os.path.join(graph_data_path, category_folder, f"{filename}.edgelist")
#             if not os.path.exists(graph_path):
#                 pbar.update(1)
#                 continue

#             try:
#                 # Efficient edge list reading
#                 with open(graph_path, 'r') as f:
#                     edge_list = [tuple(map(int, line.strip().split()))
#                                  for line in f if line.strip() and not line.startswith('#')]
#                 if not edge_list:
#                     pbar.update(1)
#                     continue

#                 edge_index = torch.tensor(edge_list).t().contiguous()
#                 num_nodes = int(edge_index.max().item()) + 1

#                 # Build networkx graph and calculate node features
#                 G_nx = nx.DiGraph()
#                 G_nx.add_nodes_from(range(num_nodes))
#                 G_nx.add_edges_from(edge_list)
#                 x = calculate_node_features(G_nx, num_nodes)

#                 data = Data(
#                     x=x,
#                     edge_index=edge_index,
#                     y=torch.tensor([LABEL_MAPPING[class_name]], dtype=torch.long)
#                 )
#                 data_list.append(data)
#             except Exception as e:
#                 print(f"[Warning] Error processing {graph_path}: {str(e)}")
#             finally:
#                 pbar.update(1)
#     return data_list

# def visualize_attributes(attribute_matrix):
#     attr_vectors = attribute_matrix().detach().cpu().numpy()  # Detach the tensor to remove gradients
#     tsne = TSNE(n_components=2, random_state=42, perplexity=3)  # Set perplexity to 3 or another value less than 5
#     reduced = tsne.fit_transform(attr_vectors)
#     plt.scatter(reduced[:, 0], reduced[:, 1], c=range(len(LABEL_MAPPING)), cmap='viridis')
#     for i, label in enumerate(LABEL_MAPPING.keys()):
#         plt.annotate(label, (reduced[i, 0], reduced[i, 1]))
#     plt.title("t-SNE Visualization of Attribute Vectors")
#     plt.show()

# # --- Load datasets ---
# print("Loading datasets...")
# train_data_raw = load_graphs(train_graphs, desc="Training graphs")
# val_data_raw = load_graphs(val_graphs, desc="Validation graphs")
# test_data_raw = load_graphs(test_graphs, desc="Test graphs")

# # Filter out the unseen class ('addisplay') from training data
# unseen_label = LABEL_MAPPING['addisplay']
# train_data = [d for d in train_data_raw if d.y.item() != unseen_label]
# val_data = val_data_raw
# test_data = test_data_raw

# from gensim.models import KeyedVectors

# # --- Load GloVe embeddings (pre-converted to Word2Vec format) ---
# glove_model = KeyedVectors.load_word2vec_format('/content/model.word2vec', binary=False)

# # Otherwise, try loading directly (GloVe format):
# # glove_model = KeyedVectors.load_word2vec_format('path/to/glove.6B.100d.txt', binary=False)
# # --- Build a dictionary of pretrained embeddings for each label ---
# pretrained_embeddings = {}
# for label in LABEL_MAPPING.keys():
#     if label in glove_model:
#         pretrained_embeddings[label] = glove_model[label].tolist()
#     else:
#         # If the label isn't found, create a random 100-d vector.
#         pretrained_embeddings[label] = [random.uniform(-0.1, 0.1) for _ in range(100)]

# import torch
# import torch.nn as nn  # <-- add this at the top
# import torch.nn.functional as F

# # --- Attribute matrix module ---
# # --- Updated Attribute Matrix Module ---
# class AttributeMatrix(nn.Module):
#     def __init__(self, num_classes, input_dim, embed_dim, label_mapping, pretrained_embeddings):
#         super().__init__()
#         self.embed_dim = embed_dim

#         # Enhanced projection network with more layers
#         self.proj = nn.Sequential(
#             nn.Linear(input_dim, input_dim),
#             nn.LeakyReLU(0.2),
#             nn.Linear(input_dim, embed_dim),
#             nn.LeakyReLU(0.2),
#             nn.Linear(embed_dim, embed_dim)
#         )

#         # Separate residual path
#         self.residual = nn.Linear(input_dim, embed_dim)

#         # Initialize embeddings with higher weight for trojan class
#         embeddings_list = []
#         for label, idx in sorted(label_mapping.items(), key=lambda x: x[1]):
#             if label in pretrained_embeddings:
#                 vec = torch.tensor(pretrained_embeddings[label], dtype=torch.float32)

#                 # Give more weight to the trojan class embeddings
#                 if label == 'trojan':
#                     vec = vec * 1.2  # Amplify trojan features

#             else:
#                 vec = torch.randn(input_dim)
#             embeddings_list.append(vec)

#         attributes_tensor = torch.stack(embeddings_list, dim=0)
#         projected = self.proj(attributes_tensor)
#         residual_out = self.residual(attributes_tensor)
#         combined = projected + residual_out  # Residual connection

#         # Initialize the attributes with the combined embeddings
#         self.attributes = nn.Parameter(combined)

#     def forward(self, indices=None):
#         # Add L2 normalization for better cosine similarity calculation
#         attr = F.normalize(self.attributes, p=2, dim=1)
#         return attr if indices is None else attr[indices]

# # --- Zero-Shot GNN Model ---
# class ZSLGNN(nn.Module):
#     def __init__(self, input_dim=7, hidden_dim=64, embed_dim=64, dropout_rate=0.3):
#         super().__init__()
#         # Increase attention heads for better feature attention
#         self.conv1 = GATConv(input_dim, hidden_dim, heads=4, dropout=dropout_rate)
#         self.conv2 = GATConv(hidden_dim * 4, hidden_dim, heads=4, dropout=dropout_rate)
#         self.conv3 = GATConv(hidden_dim * 4, hidden_dim, heads=4, dropout=dropout_rate)

#         # Add batch normalization for more stable training
#         self.bn1 = nn.BatchNorm1d(hidden_dim * 4)
#         self.bn2 = nn.BatchNorm1d(hidden_dim * 4)
#         self.bn3 = nn.BatchNorm1d(hidden_dim * 4)

#         # Enhanced MLP for final projection
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_dim * 4, hidden_dim * 2),
#             nn.LeakyReLU(0.2),
#             nn.Dropout(dropout_rate),
#             nn.Linear(hidden_dim * 2, embed_dim),
#             nn.LeakyReLU(0.2),
#         )
#         self.dropout = nn.Dropout(dropout_rate)

#     def forward(self, data):
#         x = data.x.to(device)
#         edge_index = data.edge_index.to(device)
#         batch = data.batch.to(device) if hasattr(data, 'batch') else torch.zeros(x.size(0), dtype=torch.long, device=device)

#         # Forward pass with skip connections and batch normalization
#         x1 = self.dropout(F.elu(self.conv1(x, edge_index)))
#         x1 = self.bn1(x1)

#         x2 = self.dropout(F.elu(self.conv2(x1, edge_index)))
#         x2 = self.bn2(x2)

#         x3 = torch.cat([x1, x2], dim=1)
#         x3 = self.dropout(F.elu(self.conv3(x3, edge_index)))
#         x3 = self.bn3(x3)

#         # Global pooling with attention to important nodes
#         x_pool = global_mean_pool(x3, batch)

#         # Final projection
#         out = self.fc(x_pool)
#         return out

# # --- Set up model, optimizer, scheduler, and attribute matrix ---
# model = ZSLGNN(input_dim=7).to(device)
# attribute_matrix = AttributeMatrix(
#     num_classes=len(LABEL_MAPPING),
#     input_dim=100,        # dimension of your pretrained GloVe embeddings
#     embed_dim=64,
#     label_mapping=LABEL_MAPPING,
#     pretrained_embeddings=pretrained_embeddings
# ).to(device)

# # Visualize attributes after defining attribute_matrix
# visualize_attributes(attribute_matrix)

# from torch.optim import AdamW
# from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
# # Use separate learning rates for model and attribute matrix
# model_params = list(model.parameters())
# attr_params = list(attribute_matrix.parameters())

# optimizer = AdamW([
#     {'params': model_params, 'lr': 0.001, 'weight_decay': 0.01},
#     {'params': attr_params, 'lr': 0.002, 'weight_decay': 0.005}  # Higher learning rate for attribute matrix
# ])

# # Use a more gradual scheduler
# scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.0001)

# # --- Data loaders ---
# train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
# val_loader = DataLoader(val_data, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)
# test_loader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# # --- Training function ---
# def train(model, loader, optimizer, scheduler, attribute_matrix):
#     model.train()
#     total_loss = 0.0

#     # Adjust class weights to emphasize trojan (class 3)
#     class_weights = torch.tensor([1.0, 1.0, 1.0, 1.2, 1.5]).to(device)  # Increased weight for trojan

#     margin = 0.3  # Increase margin for better separation

#     for data in tqdm(loader, desc="Training"):
#         data = data.to(device)
#         optimizer.zero_grad()
#         embeddings = model(data)

#         # Get labels and corresponding attribute targets
#         y_labels = data.y.squeeze() if data.y.dim() > 1 else data.y
#         targets = attribute_matrix(y_labels)

#         # Normalize embeddings and targets
#         embeddings_norm = F.normalize(embeddings, p=2, dim=1)
#         targets_norm = targets  # Already normalized

#         # Cosine distance per sample
#         cosine_loss = 1 - F.cosine_similarity(embeddings_norm, targets_norm, dim=1)

#         # A larger margin-based term for better separation
#         margin_loss = torch.clamp(margin - F.cosine_similarity(embeddings_norm, targets_norm, dim=1), min=0.0)

#         # Add focal loss component to focus on hard examples
#         focal_weight = torch.exp(-F.cosine_similarity(embeddings_norm, targets_norm, dim=1))
#         loss_elements = (cosine_loss + margin_loss) * focal_weight

#         # Apply class weighting
#         weighted_loss = loss_elements * class_weights[y_labels]

#         # Add regularization term
#         loss = weighted_loss.mean() + 0.01 * torch.norm(embeddings, p=2, dim=1).mean()

#         loss.backward()

#         # Gradient clipping to stabilize training
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

#         optimizer.step()
#         total_loss += loss.item()

#     scheduler.step()
#     return total_loss / len(loader)

# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score, f1_score

# # --- Evaluation function ---
# @torch.no_grad()
# def evaluate(model, loader, attribute_matrix, bias=0.):
#     model.eval()
#     y_true, y_pred = [], []

#     # Adjust class biases to correct for prediction imbalance
#     # Increase bias for trojan (index 3) to make it more likely to be predicted
#     class_biases = torch.zeros(len(LABEL_MAPPING)).to(device)
#     class_biases[0] = 0.1  # Add bias to adware
#     class_biases[1] = 0.1  # Add bias to benign
#     class_biases[2] = 0.1  # Add bias to downloader
#     # Leave trojan without additional bias
#     class_biases[4] = 0.1  # Add bias to addisplay

#     for data in tqdm(loader, desc="Evaluating"):
#         data = data.to(device)
#         embeddings = model(data)
#         y_labels = data.y.squeeze() if data.y.dim() > 1 else data.y
#         embeddings_norm = F.normalize(embeddings, p=2, dim=1)
#         attr_norm = attribute_matrix()  # Full attribute matrix

#         # Compute cosine similarity with class-specific bias adjustment
#         sims = torch.mm(embeddings_norm, attr_norm.t())

#         # Apply class-specific biases
#         sims = sims + class_biases

#         # Get predictions
#         preds = sims.argmax(dim=1)

#         y_true.extend(y_labels.cpu().tolist())
#         y_pred.extend(preds.cpu().tolist())

#     cm = confusion_matrix(y_true, y_pred)
#     print("Confusion Matrix:")
#     print(cm)
#     # Create a list of class names ordered by their numerical indices
#     ordered_class_names = [k for k, v in sorted(LABEL_MAPPING.items(), key=lambda item: item[1])]
#     disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=ordered_class_names)

#     disp.plot(cmap=plt.cm.Blues)
#     plt.title("Confusion Matrix")
#     plt.show()

#     accuracy = accuracy_score(y_true, y_pred)
#     f1 = f1_score(y_true, y_pred, average='macro')
#     print(classification_report(y_true, y_pred))

#     return accuracy, f1

# # --- Main training loop ---
# if __name__ == "__main__":
#     print(f"Using device: {device}")

#     EPOCHS = 30
#     best_val_acc = 0.0

#     for epoch in range(EPOCHS):
#         train_loss = train(model, train_loader, optimizer, scheduler, attribute_matrix)
#         print(f"Epoch {epoch + 1}/{EPOCHS}: Loss = {train_loss:.4f}")

#         # Evaluate on validation set every 3 epochs
#         if (epoch + 1) % 3 == 0:
#             val_acc, val_f1 = evaluate(model, val_loader, attribute_matrix)
#             print(f"Validation Accuracy: {val_acc:.4f}, Macro F1: {val_f1:.4f}")
#             if val_acc > best_val_acc:
#                 best_val_acc = val_acc
#                 # Save best model checkpoint
#                 torch.save({
#                     'model_state_dict': model.state_dict(),
#                     'attribute_matrix_state_dict': attribute_matrix.state_dict(),
#                     'optimizer_state_dict': optimizer.state_dict()
#                 }, 'best_model.pth')

#     print("\nFinal Test Evaluation:")
#     test_acc, test_f1 = evaluate(model, test_loader, attribute_matrix)
#     print(f"Test Accuracy: {test_acc:.4f}, Macro F1: {test_f1:.4f}")


#     # Play audio after code execution
#     youtube_url = "https://www.youtube.com/watch?v=j9TykAvgnC4"
#     display(play_youtube_audio(youtube_url))




ModuleNotFoundError: No module named 'torch_geometric'

In [4]:
!pip install spektral

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 7.7 MB/s eta 0:00:00


In [5]:
!pip install yt-dlp ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.2 MB/s eta 0:00:00


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_file = '/content/glove.6B.100d.txt' # Replace with the>
word2vec_file = 'model.word2vec'
glove2word2vec(glove_file, word2vec_file)
print("Conversion complete")

<ipython-input-5-1c155e8a7641>:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_file)


Conversion complete


In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm import tqdm
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
from sklearn.metrics import classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import yt_dlp
import IPython.display as ipd

def play_youtube_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': 'audio.%(ext)s'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return ipd.Audio("audio.mp3", autoplay=True)

# --- Set seeds and device ---
SEED = 75
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    # Enable benchmark for faster training when possible
    torch.backends.cudnn.benchmark = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# --- Label and path definitions ---
LABEL_MAPPING = {
    'adware': 0,
    'benign': 1,
    'downloader': 2,
    'trojan': 3,
    'addisplay': 4  # This class will be treated as unseen during training
}

# Set file paths (modify if needed)
graph_data_path = '/content/extracted_files/malnet/malnet-graphs-tiny'
split_info_path = '/content/extracted_files/splitinfo/split_info_tiny/type'

# --- Split load helper ---
def load_split(file_name):
    split_file = os.path.join(split_info_path, file_name)
    with open(split_file, 'r') as f:
        return [line.strip() for line in f]

train_graphs = load_split('train.txt')
val_graphs = load_split('val.txt')
test_graphs = load_split('test.txt')

# --- Node feature calculation ---
@torch.no_grad()
def calculate_node_features(G_nx, num_nodes):
    # Basic centrality measures
    degrees = torch.tensor([d for _, d in G_nx.degree()], dtype=torch.float32)
    G_undirected = G_nx.to_undirected()
    clustering_dict = nx.clustering(G_undirected)
    clustering_values = torch.tensor([clustering_dict[i] for i in range(num_nodes)], dtype=torch.float32)

    # Calculate additional centrality metrics with size-based approximation
    if num_nodes > 500:
        betweenness_dict = nx.betweenness_centrality(G_nx, k=min(50, num_nodes))
        closeness_dict = {i: 0.5 for i in range(num_nodes)}  # Default value for large graphs
    else:
        betweenness_dict = nx.betweenness_centrality(G_nx)
        try:
            closeness_dict = nx.closeness_centrality(G_nx)
        except:
            closeness_dict = {i: 0.5 for i in range(num_nodes)}

    betweenness_values = torch.tensor([betweenness_dict[i] for i in range(num_nodes)], dtype=torch.float32)
    closeness_values = torch.tensor([closeness_dict[i] for i in range(num_nodes)], dtype=torch.float32)

    # Calculate eigenvector centrality (important for capturing influence patterns)
    try:
        eigenvector_dict = nx.eigenvector_centrality(G_nx, max_iter=100)
    except Exception as e:
        eigenvector_dict = {i: 1.0 / num_nodes for i in range(num_nodes)}
    eigenvector_values = torch.tensor([eigenvector_dict[i] for i in range(num_nodes)], dtype=torch.float32)

    # Calculate pagerank (useful for identifying important nodes)
    try:
        pagerank_dict = nx.pagerank(G_nx, max_iter=100)
    except:
        pagerank_dict = {i: 1.0 / num_nodes for i in range(num_nodes)}
    pagerank_values = torch.tensor([pagerank_dict[i] for i in range(num_nodes)], dtype=torch.float32)

    # Include graph density feature (trojans often have distinct density patterns)
    graph_density = nx.density(G_nx)
    density_feature = torch.full((num_nodes, 1), graph_density, dtype=torch.float32)

    # Add spectral features
    try:
        laplacian = nx.normalized_laplacian_matrix(G_undirected)
        eigenvalues = np.linalg.eigvals(laplacian.toarray())[:5]  # Top 5 eigenvalues
        spectral_feature = torch.tensor([[np.mean(eigenvalues)]] * num_nodes, dtype=torch.float32)
    except:
        spectral_feature = torch.zeros((num_nodes, 1), dtype=torch.float32)

    # Combine all features
    return torch.cat([
        degrees.unsqueeze(1),
        clustering_values.unsqueeze(1),
        betweenness_values.unsqueeze(1),
        eigenvector_values.unsqueeze(1),
        closeness_values.unsqueeze(1),
        pagerank_values.unsqueeze(1),
        density_feature,
        spectral_feature
    ], dim=1)

# --- Graph loading function with improved error handling ---
def load_graphs(graph_list, desc="Loading graphs"):
    data_list = []
    with tqdm(total=len(graph_list), desc=desc, position=0, leave=True) as pbar:
        for graph_entry in graph_list:
            # Expecting "class/filename" structure
            category_folder, filename = os.path.split(graph_entry)
            class_name = category_folder.split('/')[0]
            if class_name not in LABEL_MAPPING:
                pbar.update(1)
                continue

            graph_path = os.path.join(graph_data_path, category_folder, f"{filename}.edgelist")
            if not os.path.exists(graph_path):
                pbar.update(1)
                continue

            try:
                # Efficient edge list reading
                with open(graph_path, 'r') as f:
                    edge_list = [tuple(map(int, line.strip().split()))
                                 for line in f if line.strip() and not line.startswith('#')]
                if not edge_list:
                    pbar.update(1)
                    continue

                edge_index = torch.tensor(edge_list).t().contiguous()
                num_nodes = int(edge_index.max().item()) + 1

                # Build networkx graph and calculate node features
                G_nx = nx.DiGraph()
                G_nx.add_nodes_from(range(num_nodes))
                G_nx.add_edges_from(edge_list)
                x = calculate_node_features(G_nx, num_nodes)

                data = Data(
                    x=x,
                    edge_index=edge_index,
                    y=torch.tensor([LABEL_MAPPING[class_name]], dtype=torch.long)
                )
                data_list.append(data)
            except Exception as e:
                print(f"[Warning] Error processing {graph_path}: {str(e)}")
            finally:
                pbar.update(1)
    return data_list

def visualize_attributes(attribute_matrix):
    attr_vectors = attribute_matrix().detach().cpu().numpy()  # Detach the tensor to remove gradients
    tsne = TSNE(n_components=2, random_state=42, perplexity=3)  # Set perplexity to 3 or another value less than 5
    reduced = tsne.fit_transform(attr_vectors)
    plt.scatter(reduced[:, 0], reduced[:, 1], c=range(len(LABEL_MAPPING)), cmap='viridis')
    for i, label in enumerate(LABEL_MAPPING.keys()):
        plt.annotate(label, (reduced[i, 0], reduced[i, 1]))
    plt.title("t-SNE Visualization of Attribute Vectors")
    plt.show()

# --- Load datasets ---
print("Loading datasets...")
train_data_raw = load_graphs(train_graphs, desc="Training graphs")
val_data_raw = load_graphs(val_graphs, desc="Validation graphs")
test_data_raw = load_graphs(test_graphs, desc="Test graphs")

# Filter out the unseen class ('addisplay') from training data
unseen_label = LABEL_MAPPING['addisplay']
train_data = [d for d in train_data_raw if d.y.item() != unseen_label]
val_data = val_data_raw
test_data = test_data_raw

from gensim.models import KeyedVectors

# --- Load GloVe embeddings (pre-converted to Word2Vec format) ---
glove_model = KeyedVectors.load_word2vec_format('/content/model.word2vec', binary=False)

# Otherwise, try loading directly (GloVe format):
# glove_model = KeyedVectors.load_word2vec_format('path/to/glove.6B.100d.txt', binary=False)
# --- Build a dictionary of pretrained embeddings for each label ---
pretrained_embeddings = {}
for label in LABEL_MAPPING.keys():
    if label in glove_model:
        pretrained_embeddings[label] = glove_model[label].tolist()
    else:
        # If the label isn't found, create a random 100-d vector.
        pretrained_embeddings[label] = [random.uniform(-0.1, 0.1) for _ in range(100)]

import torch
import torch.nn as nn  # <-- add this at the top
import torch.nn.functional as F

# --- Attribute matrix module ---
# --- Updated Attribute Matrix Module ---
class AttributeMatrix(nn.Module):
    def __init__(self, num_classes, input_dim, embed_dim, label_mapping, pretrained_embeddings):
        super().__init__()
        self.embed_dim = embed_dim

        # Enhanced projection network with more layers
        self.proj = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(input_dim, embed_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(embed_dim, embed_dim)
        )

        # Separate residual path
        self.residual = nn.Linear(input_dim, embed_dim)

        # Initialize embeddings with balanced weights
        embeddings_list = []
        for label, idx in sorted(label_mapping.items(), key=lambda x: x[1]):
            if label in pretrained_embeddings:
                vec = torch.tensor(pretrained_embeddings[label], dtype=torch.float32)

                # Give more weight to the trojan class embeddings but keep it balanced
                if label == 'trojan':
                    vec = vec * 1.05  # Reduced amplification for trojan features

            else:
                vec = torch.randn(input_dim)
            embeddings_list.append(vec)

        attributes_tensor = torch.stack(embeddings_list, dim=0)
        projected = self.proj(attributes_tensor)
        residual_out = self.residual(attributes_tensor)
        combined = projected + residual_out  # Residual connection

        # Initialize the attributes with the combined embeddings
        self.attributes = nn.Parameter(combined)

    def forward(self, indices=None):
        # Add L2 normalization for better cosine similarity calculation
        attr = F.normalize(self.attributes, p=2, dim=1)
        return attr if indices is None else attr[indices]

# --- Zero-Shot GNN Model ---
class ZSLGNN(nn.Module):
    def __init__(self, input_dim=7, hidden_dim=64, embed_dim=64, dropout_rate=0.3):
        super().__init__()
        # Increase attention heads for better feature attention
        self.conv1 = GATConv(input_dim, hidden_dim, heads=4, dropout=dropout_rate)
        self.conv2 = GATConv(hidden_dim * 4, hidden_dim, heads=4, dropout=dropout_rate)
        self.conv3 = GATConv(hidden_dim * 4 * 2, hidden_dim, heads=4, dropout=dropout_rate)

        # Node-level attention
        self.node_attention = nn.Sequential(
            nn.Linear(hidden_dim * 4, 1),
            nn.Sigmoid()
        )

        # Add batch normalization for more stable training
        self.bn1 = nn.BatchNorm1d(hidden_dim * 4)
        self.bn2 = nn.BatchNorm1d(hidden_dim * 4)
        self.bn3 = nn.BatchNorm1d(hidden_dim * 4)

        # Enhanced MLP for final projection
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 4, hidden_dim * 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim * 2, embed_dim),
            nn.LeakyReLU(0.2),
        )
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, data):
        x = data.x.to(device)
        # Apply feature normalization
        x = (x - x.mean(dim=0)) / (x.std(dim=0) + 1e-6)

        edge_index = data.edge_index.to(device)
        batch = data.batch.to(device) if hasattr(data, 'batch') else torch.zeros(x.size(0), dtype=torch.long, device=device)

        # Forward pass with skip connections and batch normalization
        x1 = self.dropout(F.elu(self.conv1(x, edge_index)))
        x1 = self.bn1(x1)

        x2 = self.dropout(F.elu(self.conv2(x1, edge_index)))
        x2 = self.bn2(x2)

        x3 = torch.cat([x1, x2], dim=1)
        x3_res = self.dropout(F.elu(self.conv3(x3, edge_index)))
        # Residual connection with dimension matching
        x3 = x3_res + x3[:, :x3_res.size(1)]
        x3 = self.bn3(x3)

        # Apply node attention
        attention_weights = self.node_attention(x3)
        x3 = x3 * attention_weights

        # Global pooling
        x_pool = global_mean_pool(x3, batch)

        # Final projection
        out = self.fc(x_pool)
        return out


# --- Set up model, optimizer, scheduler, and attribute matrix ---
# Change this line
model = ZSLGNN(input_dim=7).to(device)
attribute_matrix = AttributeMatrix(
    num_classes=len(LABEL_MAPPING),
    input_dim=100,        # dimension of your pretrained GloVe embeddings
    embed_dim=64,
    label_mapping=LABEL_MAPPING,
    pretrained_embeddings=pretrained_embeddings
).to(device)

# Visualize attributes after defining attribute_matrix
visualize_attributes(attribute_matrix)

from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
# Use separate learning rates for model and attribute matrix
model_params = list(model.parameters())
attr_params = list(attribute_matrix.parameters())

optimizer = AdamW([
    {'params': model_params, 'lr': 0.001, 'weight_decay': 0.01},
    {'params': attr_params, 'lr': 0.002, 'weight_decay': 0.005}  # Higher learning rate for attribute matrix
])

# Use a more gradual scheduler
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.0001)

# --- Data loaders ---
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# --- Training function ---
def train(model, loader, optimizer, scheduler, attribute_matrix):
    model.train()
    total_loss = 0.0

    # Balanced class weights
    class_weights = torch.tensor([1.0, 1.0, 1.0, 1.2, 1.5]).to(device)

    # Margin for better separation
    margin = 0.3

    for data in tqdm(loader, desc="Training"):
        data = data.to(device)
        optimizer.zero_grad()
        embeddings = model(data)

        # Get labels and corresponding attribute targets
        y_labels = data.y.squeeze() if data.y.dim() > 1 else data.y
        targets = attribute_matrix(y_labels)

        # Normalize embeddings and targets
        embeddings_norm = F.normalize(embeddings, p=2, dim=1)
        targets_norm = targets  # Already normalized

        # Cosine distance per sample
        cosine_loss = 1 - F.cosine_similarity(embeddings_norm, targets_norm, dim=1)

        # Margin-based term for better separation
        margin_loss = torch.clamp(margin - F.cosine_similarity(embeddings_norm, targets_norm, dim=1), min=0.0)

        # Focal loss component to focus on hard examples
        focal_weight = torch.exp(-F.cosine_similarity(embeddings_norm, targets_norm, dim=1))
        loss_elements = (cosine_loss + margin_loss) * focal_weight

        # Apply class weighting
        weighted_loss = loss_elements * class_weights[y_labels]

        # Add regularization term
        loss = weighted_loss.mean() + 0.01 * torch.norm(embeddings, p=2, dim=1).mean()

        loss.backward()

        # Gradient clipping to stabilize training
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item()

    scheduler.step()
    return total_loss / len(loader)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score, f1_score

# --- Evaluation function ---
@torch.no_grad()
def evaluate(model, loader, attribute_matrix, epoch=0):
    model.eval()
    y_true, y_pred = []
    class_correct = {i: 0 for i in range(len(LABEL_MAPPING))}
    class_total = {i: 0 for i in range(len(LABEL_MAPPING))}

    # Dynamic temperature annealing
    temperature = max(1.0, 1.5 - (epoch * 0.05))

    # Balance class biases - adjust for addisplay and trojan
    class_biases = torch.zeros(len(LABEL_MAPPING)).to(device)
    class_biases[0] = 0.08  # Adware
    class_biases[1] = 0.08  # Benign
    class_biases[2] = 0.05  # Downloader (already performs well)
    class_biases[3] = 0.05  # Small boost for trojan
    class_biases[4] = 0.15  # Higher bias for addisplay to improve detection

    for data in tqdm(loader, desc="Evaluating"):
        data = data.to(device)
        embeddings = model(data)
        y_labels = data.y.squeeze() if data.y.dim() > 1 else data.y
        embeddings_norm = F.normalize(embeddings, p=2, dim=1)
        attr_norm = attribute_matrix()

        # Apply temperature scaling before adding biases
        sims = torch.mm(embeddings_norm, attr_norm.t()) / temperature

        # Apply class biases after temperature scaling
        sims = sims + class_biases

        # Get predictions
        preds = sims.argmax(dim=1)

        # Track class-wise performance
        for i in range(len(y_labels)):
            label = y_labels[i].item()
            pred = preds[i].item()
            class_total[label] += 1
            if pred == label:
                class_correct[label] += 1

        y_true.extend(y_labels.cpu().tolist())
        y_pred.extend(preds.cpu().tolist())

    # Display class-wise accuracy
    print("\nClass-wise accuracy:")
    for label_name, label_idx in LABEL_MAPPING.items():
        accuracy = class_correct[label_idx] / max(1, class_total[label_idx])
        print(f"{label_name}: {accuracy:.2f}")

    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Display confusion matrix
    ordered_class_names = [k for k, v in sorted(LABEL_MAPPING.items(), key=lambda item: item[1])]
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=ordered_class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    print(classification_report(y_true, y_pred))

    return accuracy, f1

# --- Main training loop ---
if __name__ == "__main__":
    print(f"Using device: {device}")

    # Training parameters
    EPOCHS = 30
    patience = 5
    patience_counter = 0
    best_val_acc = 0.0
    best_epoch = 0

    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, scheduler, attribute_matrix)
        print(f"Epoch {epoch + 1}/{EPOCHS}: Loss = {train_loss:.4f}")

        # Evaluate more frequently
        if (epoch + 1) % 2 == 0:  # Every 2 epochs
            val_acc, val_f1 = evaluate(model, val_loader, attribute_matrix, epoch=epoch)
            print(f"Validation Accuracy: {val_acc:.4f}, Macro F1: {val_f1:.4f}")

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_epoch = epoch
                patience_counter = 0

                # Save best model checkpoint
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'attribute_matrix_state_dict': attribute_matrix.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': epoch,
                    'val_acc': val_acc,
                    'val_f1': val_f1
                }, 'best_model.pth')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}. Best epoch was {best_epoch+1} with validation accuracy {best_val_acc:.4f}")
                    break

    # Load best model for final evaluation
    checkpoint = torch.load('best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    attribute_matrix.load_state_dict(checkpoint['attribute_matrix_state_dict'])

    print("\nFinal Test Evaluation:")
    test_acc, test_f1 = evaluate(model, test_loader, attribute_matrix)
    print(f"Test Accuracy: {test_acc:.4f}, Macro F1: {test_f1:.4f}")

    # Play audio after code execution
    youtube_url = "https://www.youtube.com/watch?v=j9TykAvgnC4"
    display(play_youtube_audio(youtube_url))




Using device: cuda
Loading datasets...


Training graphs:   0%|          | 0/3500 [00:00<?, ?it/s]<ipython-input-6-642e9935f61f>:114: ComplexWarning: Casting complex values to real discards the imaginary part
  spectral_feature = torch.tensor([[np.mean(eigenvalues)]] * num_nodes, dtype=torch.float32)
Training graphs:   1%|          | 25/3500 [01:41<4:52:30,  5.05s/it]